#### Mi az a Mixin?

A mixin osztályok olyan osztályok, amelyeket a legtöbb esetben nem lehet önmagukban használni.<br>
A céljuk kiegészítő funkciók nyújtása a meglévő osztályokhoz.

#### Egy egszerű példa

In [2]:
class Allat:
    def __init__(self, nev, hang):
        self.nev = nev
        self.hang = hang

    def eszik(self):
        print(f"{self.nev} eszik.")


# A mixin példa,
class HangoskodoMixin:
    def koszont(self):
        print(f"{self.nev} azt mondja: {self.hang}")


class Kutya(Allat, HangoskodoMixin):
    def __init__(self, nev):
        super().__init__(nev, "Vau!")


class Macska(Allat, HangoskodoMixin):
    def __init__(self, nev):
        super().__init__(nev, "Miau!")


# Használat
bodri = Kutya("Riki")
lujzi = Macska("Lujzi")

bodri.koszont()
lujzi.eszik()
lujzi.koszont()

Riki azt mondja: Vau!
Lujzi eszik.
Lujzi azt mondja: Miau!


#### MRO – Method Resolution Order

-> Amikor több osztálytól öröklünk Python-ban, akkor balról jobbra "keresi" a metódusokat a Python.
Máshogy fogalmazva a bal oldali osztálynak van magasabb precedenciája minden esetben.

In [5]:
# Szerencsére könnyen meg tudjuk nézni az MRO-t adott osztályhoz az __mro__ dunder-el.
print(Kutya.__mro__)

(<class '__main__.Kutya'>, <class '__main__.Allat'>, <class '__main__.HangoskodoMixin'>, <class 'object'>)


In [ ]:
# Egy érdekes példa:
from abc import ABC, abstractmethod


class ExampleA(ABC):
    def method_a(self):
        print("ExampleA.method_a() meghívva – ez a konkrét (nem absztrakt) metódus.")

    @abstractmethod
    def method_b(self):
        """Ezt a metódust az alosztálynak kell megvalósítania."""
        pass


class ExampleB(ABC):
    def method_b(self):
        print("ExampleB.method_b() meghívva – ez a konkrét (nem absztrakt) metódus.")

    @abstractmethod
    def method_a(self):
        """Ezt a metódust az alosztálynak kell megvalósítania."""
        pass


class Laci(ExampleA, ExampleB):
    pass

In [ ]:
## Mi fog történni?
# laci = Laci()

Magyarázat:
Az MRO szerint a Python először nézi az ExampleA-t, ahol method_b() absztrakt marad.

ExampleB megoldja ugyan method_b()-t, de az MRO miatt a Python nem látja ezt “kitöltve”, és még mindig úgy értékeli, hogy van absztrakt metódus → TypeError: Can't instantiate abstract class

In [ ]:
class LaciFixed(ExampleA, ExampleB):
    def method_a(self):
        return ExampleA.method_a(self)  # vagy saját implementáció

    def method_b(self):
        return ExampleB.method_b(self)  # vagy saját implementáció


laci_jav = LaciFixed()
laci_jav.method_a()
laci_jav.method_b()

ExampleA.method_a() meghívva – ez a konkrét (nem absztrakt) metódus.
ExampleB.method_b() meghívva – ez a konkrét (nem absztrakt) metódus.


In [ ]:
# Egy másik példa, de most Mixin ütközés
class MixinA:
    def greet(self):
        print("Hello from MixinA")


class MixinB:
    def greet(self):
        print("Hello from MixinB")


class MyClass(MixinA, MixinB):
    pass


obj = MyClass()
obj.greet()  # Hello from MixinA
# Magyarázat: MixinA-nak magasabb a precedenciája, mert ez van baloldalon...

Hello from MixinA


In [ ]:
class X:
    def process(self):
        print("X")


class Y(X):
    def process(self):
        print("Y")
        super().process()


class Z(X):
    def process(self):
        print("Z")
        super().process()


class Combined(Y, Z):
    def process(self):
        print("Combined")
        super().process()


c = Combined()
c.process()
# Magyarázat: Itt jól látszik, hogy a super kulcsszó a resolution order következp tagját hívja meg mindig.
# resolution order itt: Combined.__mro__ : (__main__.Combined, __main__.Y, __main__.Z, __main__.X, object)
# Először a Combiend process-je fut, utána az Y mert baoldalon van, utána Z és legvégül X.

Combined
Y
Z
X


#### Django példa majd későbbre

```python

# .... Valami ilyesmi view osztályunk lenne...
class MyView(LoginRequiredMixin, DetailView):
# itt a resoulution order: MyView → LoginRequiredMixin → DetailView → ... → object


# A LoginRequiredMixin kb így néz ki...

class LoginRequiredMixin:
    def dispatch(self, request, *args, **kwargs):
        if not request.user.is_authenticated:
            return redirect_to_login(request.get_full_path())
        return super().dispatch(request, *args, **kwargs)

```

A lényeg itt a dispatch metódus:
<p>-> A Django osztály alapú nézetekben (CBV) minden kérés feldolgozása a dispatch() metóduson keresztül történik.
Itt dől el, hogy post vagy get requesthez tartozó logikát akarunk futtatni...</p>

  - Először lefut a LoginRequiredMixin.dispatch(), elvégzi az ellenőrzést...
  - Meghívja a super().dispatch()-ot, ami bemutatott MyView.dispatch metódusára vezet.
  (Fontos látni, hogy a super a method resolution orderben a következő osztályra mutat, ami tényleg a DetailView lesz.)

Ha a sorrendet megcseréled, a mixin soha nem tudja “becsapni az ajtót” a jogosulatlan felhasználók előtt, mert a super().dispatch() már lefutott előtte.